<a href="https://colab.research.google.com/github/Ishar786/Python/blob/main/SWOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the Google AI library, requests for web access, BeautifulSoup for HTML parsing, and lxml for speed
!pip install -q -U google-generativeai requests beautifulsoup4 lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 20.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.4 which is incompatible.


In [ ]:
import google.generativeai as genai
import requests
from bs4 import BeautifulSoup
import lxml
from google.colab import userdata
import textwrap
from IPython.display import Markdown

# --- CONFIGURATION (No changes) ---
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)
except userdata.SecretNotFoundError as e:
    print("API Key not found. Please follow Step 1 to add your GOOGLE_API_KEY to Colab Secrets.")
    raise e
model = genai.GenerativeModel('gemini-1.5-flash')

# --- MODULE 1: WEB SCRAPER (No changes) ---
# ... [Keeping this function the same as before, collapsed for brevity] ...
def get_article_text(url: str) -> str:
    print(f"Scraping URL: {url}...")
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'lxml')
        for script_or_style in soup(["script", "style"]):
            script_or_style.decompose()
        main_content = soup.find('article') or soup.find('main') or soup.body
        text = main_content.get_text(separator='\n', strip=True)
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        cleaned_text = '\n'.join(chunk for chunk in chunks if chunk)
        print("Scraping successful.")
        return cleaned_text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

# --- MODULE 2: AI ANALYSIS CORE (FINAL TWEAK FOR FLOW) ---
def generate_swot_point(text: str, company_name: str, swot_category: str, example: str) -> str:
    print(f"Analyzing text for {swot_category} regarding {company_name}...")

    # REFINED PROMPT with a final instruction for better narrative flow
    prompt = f"""
    You are a business analyst who MUST follow a very specific writing formula.
    Analyze the provided "Press Release Text" and generate a single paragraph for a SWOT analysis about the company '{company_name}'.

    **YOUR TASK IS TO FOLLOW THIS EXACT 5-STEP FORMULA:**

    **Step 1: The Opening Sentence.** Start with the date (month year), followed by the company name, the action (e.g., 'has acquired', 'has partnered'), the entity involved, and the immediate purpose.

    **Step 2: The Detail Sentence.** In a new sentence, provide a specific, factual detail about the entity mentioned in Step 1.

    **Step 3: The "Would" Benefit Sentence.** In a new sentence, describe the direct benefit of the action using the word "would". **You can use transitional phrases like "Through this acquisition..." to start this sentence for better flow.**

    **Step 4: The Strategic Benefit Sentence.** In another new sentence, describe the broader strategic benefit or how it helps the company's long-term goals. **You can use transitional phrases like "Further..." to start this sentence.**

    **Step 5: The Concluding Sentence.** In a final sentence, link the action to a larger company mission or goal.

    **DO NOT DEVIATE FROM THIS 5-STEP STRUCTURE. The output MUST be a single paragraph.**

    ---
    **PERFECT EXAMPLE TO MIMIC:**
    {example}
    ---

    **Press Release Text:**
    ---
    {text}
    ---
    """

    try:
        response = model.generate_content(prompt)
        print("Analysis complete.")
        return response.text.strip()
    except Exception as e:
        print(f"An error occurred during AI analysis: {e}")
        return None

# --- MODULE 3: SCALABLE SWOT FRAMEWORK (Single, best example) ---
# We will now use ONLY your best example to reinforce the template.
SWOT_EXAMPLES = {
    "Opportunity": textwrap.dedent("""
        in January 2025, Elevance has acquired Indiana University (IU) Health Plans, to enhance its local presence, expanding product offerings. IU Health Plans is a managed care organization created by Indiana University Health which provides Medicare advantage plans to 19,000 people across 36 countries. This acquisition would elevate quality and would expand Elevance's product offerings. This acquisition would strengthen the company’s efforts to cultivate healthier communities and improve health outcomes for those they are privileged to serve. This strategic step is in line with Elevance’s health equity goals and allow it to provide comprehensive access to high-quality care and timely interventions.
    """)
}

# --- Main Runner Function (No changes) ---
def analyze_url_for_swot(url: str, company_name: str, swot_category: str):
    if swot_category not in SWOT_EXAMPLES:
        print(f"Error: SWOT category '{swot_category}' is not defined.")
        return
    # Note: Using the manual text method since scraping this site is blocked.
    article_text = """
    Elevance Health is continuing its dealmaking streak, this time with the acquisition of Indiana University Health's health plan. IU Health Plans offers Medicare Advantage and commercial plans to employers and individuals in the state, and the deal includes its risk-based and medical management services. The acquisition is expected to close later this year, though financial details were not disclosed. "The acquisition of IU Health Plans will help us continue to build on our commitment to expanding access to high-quality, affordable healthcare for people in Indiana," said Felicia Norwood, executive vice president and president of government health benefits for Elevance Health, in a statement. "We look forward to the opportunity to leverage our resources and broad networks to provide IU Health Plan members with a high-value, whole-health experience." Elevance said it will work with IU Health to ensure a smooth transition for members, who will be able to continue to access IU Health's provider network. Elevance's own provider network will become available to members following the acquisition. Elevance has been on an M&A tear of late, most recently closing its acquisition of specialty pharmacy BioPlus. It also snapped up Blue Cross and Blue Shield of Louisiana in a $2.5 billion deal. The insurer has also been active in the government programs space, acquiring Integra Managed Care in New York.
    """
    example = SWOT_EXAMPLES[swot_category]
    swot_paragraph = generate_swot_point(article_text, company_name, swot_category, example)
    if swot_paragraph:
        print("\n--- Generated SWOT Paragraph ---\n")
        display(Markdown(swot_paragraph))

In [ ]:
# --- INPUTS: Paste the article text here ---

# --- ANALYSIS: Change the company name and SWOT category as needed ---

company_name = "Petrochina Company limited"
swot_category_to_find = "Opportunity"

# I have copied the text for you from the URL.
article_text_manual = """
Shell, PetroChina Sanction Phase 2 of Surat Gas Project
by Jov Onsat|Rigzone Staff | Wednesday, August 14, 2024 | 7:06 AM EST
Shell, PetroChina Sanction Phase 2 of Surat Gas Project
PetroChina and Shell agreed on an additional investment in the Surat Gas Project, their joint venture in Australia that is targeted to produce about five Tcf of natural gas over 27 years.
Image by OleksandrKalinichenko via iStock

PetroChina Co. Ltd. and Shell PLC have agreed on an additional investment in the Surat Gas Project (SGP), their joint venture in Australia that is targeted to produce about five trillion cubic feet of natural gas over 27 years.

Like phase one, which was approved by the partners 2020, phase two will supply a Shell-operated liquefaction plant on Curtis Island in the state of Queensland, Shell said in a statement. The liquefied natural gas (LNG) is meant for export, but the Surat Gas Project will also deliver gas for the domestic market.

Phase two, or SGP North, has an expected peak output of about 130 million cubic feet a day, with up to 450 production wells targeted to be drilled. Shell expects to start production 2026.

The Surat Gas Project is operated by the British energy giant and state-backed PetroChina’s 50-50 joint venture Arrow Energy Pty. Ltd.

Arrow said in a separate press release construction is scheduled to start later in 2024. The infrastructure will include a new field compression station and a 27-kilometer (16.8 miles) pipeline. Works also include upgrades for roads and other existing infrastructure.

Meanwhile phase one, which consists of over 600 wells, has an expected peak production of 90 billion feet a year according to Shell’s announcement of the final investment decision April 17, 2020.

The Surat Gas Project, which targets coal seams in the namesake basin, is part of Arrow’s 27-year agreement to supply QCLNG (QCG), the operator of the Queensland LNG facility. QCG is a joint venture operated by Shell with a 73.75 percent stake, with state-owned China National Offshore Oil Corp. and EIG Management LLC owning 25 percent and 1.25 percent respectively. Last year QCG supplied 15 percent of gas demand on Australia’s east coast according to Shell.

“QCLNG marked its 1000th cargo at the end of last year, reflecting its significance as a gas supplier for Australia and the region”, said Zoë Yujnovich, director for integrated gas and upstream at Shell. “This investment will enable us to sustain and grow this important, secure energy source that offers a lower emissions alternative to options like coal”.

Shell aims to grow its LNG production by 20–30 percent by 2030 relative to 2022, as declared in the company’s Energy Transition Strategy 2024 report.

It said in an outlook report February 14, 2024, that it expected global LNG demand to rise by over 50 percent by 2040, driven by an accelerating industrial switch from coal to gas in China and economic growth-driven demand in South and Southeast Asia.

“Shell believes LNG will play a critical role in the energy transition, replacing coal in heavy industry”, the company said in the Surat phase two news release. “It also has a continued role in displacing coal in power generation, helping to reduce local air pollution and carbon emissions. LNG helps to provide the flexibility the power system needs, at a time when renewable generation is growing rapidly”.

Shell added, “The investment in Phase 2 is expected to generate an internal rate of return in excess of the hurdle rate for Shell’s Integrated Gas business”.

Arrow chief executive Zhengxin Peng said about the additional Surat investment, “At a time when more gas is needed for homes, businesses and industries, the SGP North development will solidify Arrow’s position as a major producer of natural gas on the east coast”.

On the employment side, Arrow expects to offer up to 400 jobs for phase two.

"""

In [ ]:
# --- RUN THE ANALYSIS (using the manual text) ---

if swot_category_to_find in SWOT_EXAMPLES:
    examples = SWOT_EXAMPLES[swot_category_to_find]
    # We call the AI function directly, skipping the scraper
    swot_paragraph = generate_swot_point(article_text_manual, company_name, swot_category_to_find, examples)

    if swot_paragraph:
        print("\n--- Generated SWOT Paragraph ---\n")
        display(Markdown(swot_paragraph))
else:
    print(f"Error: SWOT category '{swot_category_to_find}' is not defined.")

Analyzing text for Opportunity regarding Petrochina Company limited...
Analysis complete.

--- Generated SWOT Paragraph ---



August 2024, PetroChina Company Limited has partnered with Shell PLC, to further develop the Surat Gas Project (SGP) in Australia.  This partnership involves an additional investment in Phase 2 of the project, projected to produce an additional 130 million cubic feet of natural gas per day at its peak.  Through this partnership, PetroChina would significantly increase its natural gas production capacity and secure long-term access to a key energy resource. Further, this strategic move strengthens PetroChina's position in the global LNG market and secures a reliable energy supply to meet growing Asian demand.  This collaboration aligns with PetroChina's broader goal of becoming a major player in the global energy transition by providing a lower-emissions energy source compared to coal.

#Refined model with weakness integrated

In [1]:
# --- IMPORTS AND CONFIGURATION ---
import google.generativeai as genai
from google.colab import userdata
import textwrap
from IPython.display import Markdown

# Configure the API Key and AI Model
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-1.5-flash')
    print("AI Model configured successfully.")
except Exception as e:
    print(f"Error during configuration: {e}")
    print("Please ensure your GOOGLE_API_KEY is set correctly in Colab Secrets.")

# --- THE STYLE GUIDE (EXAMPLES) ---
# This dictionary holds the perfect example for each SWOT category.
SWOT_EXAMPLES = {
    "Opportunity": textwrap.dedent("""
        in January 2025, Elevance has acquired Indiana University (IU) Health Plans, to enhance its local presence, expanding product offerings. IU Health Plans is a managed care organization created by Indiana University Health which provides Medicare advantage plans to 19,000 people across 36 countries. This acquisition would elevate quality and would expand Elevance's product offerings. This acquisition would strengthen the company’s efforts to cultivate healthier communities and improve health outcomes for those they are privileged to serve. This strategic step is in line with Elevance’s health equity goals and allow it to provide comprehensive access to high-quality care and timely interventions.
    """),
    "Weakness": textwrap.dedent("""
        BASF has agreed to pay a $700,000 civil penalty, to the U.S. Environmental Protection Agency (EPA) for the violation of the Toxic Substances Control Act (TSCA) related to the group’s chemical reporting obligations. According to EPA findings, BASF did not report required data on hundreds of chemical substances within the mandated timeframe, hindering the agency’s ability to assess potential public health and environmental risks. This settlement reinforces EPA’s commitment to fair but firm enforcement of environmental laws. It also serves as a reminder to the chemical industry of the importance of meeting federal reporting standards.
    """)
}

# --- THE PROMPT LIBRARY (WRITING FORMULAS) ---
# This dictionary holds the unique, step-by-step writing formula for each category.
SWOT_PROMPTS = {
    "Opportunity": """
    You are a business analyst who MUST follow a very specific writing formula to identify an OPPORTUNITY.
    Analyze the provided "Press Release Text" and generate a single paragraph.

    **YOUR TASK IS TO FOLLOW THIS EXACT 5-STEP FORMULA:**
    1.  **The Opening Sentence:** Start with the date, company name, the positive action (e.g., 'has acquired', 'has launched'), the entity involved, and the immediate purpose.
    2.  **The Detail Sentence:** Provide a specific, factual detail about the entity or initiative.
    3.  **The "Would" Benefit Sentence:** Describe the direct benefit using the word "would". Use transitional phrases like "Through this acquisition...".
    4.  **The Strategic Benefit Sentence:** Describe the broader strategic benefit. Use transitional phrases like "Further...".
    5.  **The Concluding Sentence:** Link the action to a larger company mission or goal.
    """,
    "Weakness": """
    You are a business analyst. Your task is to analyze the provided text and write a single paragraph summarizing a company's WEAKNESS, perfectly replicating the user's writing style.

    **YOUR TASK IS TO FOLLOW THIS PRECISE 3-PART STRUCTURE:**

    1.  **The Negative Event:** In the first sentence, state the date, the company, and the core negative event (e.g., "faced a lawsuit," "agreed to pay a penalty," "recalled vehicles").
    2.  **The Problem Details:** In the next sentence, provide the specific details of the problem. Use phrases like "The lawsuit alleged that...", "According to EPA findings...", or "This recall targets..." to explain the core issue.
    3.  **The Impact and Implications:** In the final sentence(s), describe the outcome. This can be the direct negative consequence for the company (e.g., financial penalties, reputational damage) OR the broader implications for the industry.
    """
}

# --- THE AI ANALYSIS ENGINE ---
# This function assembles the prompt and calls the AI model.
def generate_swot_point(text: str, company_name: str, swot_category: str) -> str:
    print(f"Analyzing text for {swot_category} regarding {company_name}...")

    # Dynamically select the correct prompt and example based on the category
    prompt_formula = SWOT_PROMPTS.get(swot_category)
    example = SWOT_EXAMPLES.get(swot_category)

    if not prompt_formula or not example:
        return f"Error: No prompt or example found for SWOT category '{swot_category}'."

    # Assemble the final prompt
    final_prompt = f"""
    {prompt_formula}

    **DO NOT DEVIATE FROM THE FORMULA. The output MUST be a single paragraph.**

    ---
    **PERFECT EXAMPLE TO MIMIC:**
    {example}
    ---

    **Press Release Text:**
    ---
    {text}
    ---
    """

    try:
        response = model.generate_content(final_prompt)
        print("Analysis complete.")
        return response.text.strip()
    except Exception as e:
        print(f"An error occurred during AI analysis: {e}")
        return None

AI Model configured successfully.


In [2]:
# --- 1. SET YOUR COMPANY NAME ---
company_name = "BASF"

# --- 2. PASTE ARTICLE TEXTS INTO THE RELEVANT BOXES ---
# You can fill one, both, or neither. The next cell will adapt automatically.

opportunity_text_input = """

China Construction Bank says it launched internal financial model based on DeepSeek R1
By Selena Li
March 28, 20253:36 PM GMT+5:30Updated March 28, 2025



Sign of China Construction Bank is seen at the 2020 China International Fair for Trade in Services (CIFTIS) in Beijing
A sign of China Construction Bank is seen at its booth at the 2020 China International Fair for Trade in Services (CIFTIS) in Beijing, China September 4, 2020. REUTERS/Tingshu Wang/File Phoyo Purchase Licensing Rights, opens new tab
March 28 - China Construction Bank (601939.SS), opens new tab said on Friday it had completed the private deployment of a large financial model based on DeepSeek R1 earlier this year.
Private deployment means for internal use only. The comments were made by the bank's CEO Zhang Yi at a press conference following annual results.
The Reuters Daily Briefing newsletter provides all the news you need to start your day. Sign up here.
Reporting by Selena Li; Editing by Aidan Lewis

Our Standards: The Thomson Reuters Trust Principles., opens new tab

Suggested Topics:









"""

weakness_text_input = """


EPA Settlement with BASF Ensures they Comply with Chemical Reporting Violations in New Jersey
May 22, 2025

Contact Information
Stephen McBay (mcbay.stephen@epa.gov)
929-243-0417
Florham Park, N.J. - The U.S. Environmental Protection Agency has finalized a settlement with international chemical company BASF Corporation resolving its violations of the Toxic Substances Control Act (TSCA). BASF has certified that it is now in full compliance with federal reporting requirements and has agreed to pay a $700,000 penalty for the past violations. This settlement is part of EPA’s core work to protect communities through chemical safety oversight.

“Chemical data reporting is a critical tool that helps EPA ensure the safety of chemicals used in communities across the country, including right here in New Jersey,” said EPA Regional Administrator Michael Martucci. “Companies are responsible for submitting accurate and timely reports. This case reflects EPA’s approach of fair but firm enforcement when those obligations are not met.”

The violations occurred at BASF’s U.S. headquarters in Florham Park, New Jersey, during the 2020 TSCA chemical reporting period. EPA found that the company did not provide information within the required timeframe about hundreds of substances that it was required to report under TSCA. These submissions help the EPA evaluate potential chemical risks and guide decisions that protect public health and the environment. BASF has corrected its submission and cooperated throughout EPA's investigation.

EPA remains committed to helping regulated facilities meet their responsibilities under TSCA and other environmental laws. Through training, technical assistance, and case-by-case problem solving, EPA works to keep communities safe and businesses thriving across New Jersey, New York, Puerto Rico, and the U.S. Virgin Islands.

Reporting requirements are critically important and widely used — not only by EPA, but by public health and policy researchers, educators, local emergency planners, state technical assistance providers, community groups, prospective home buyers, and others.

Follow EPA Region 2 on X, Instagram, and visit our Facebook page. For more information about EPA Region 2, visit our website.


"""

In [3]:
# --- This cell automatically analyzes any text provided in the cell above ---
# --- You do not need to edit this cell ---

print("--- Starting SWOT Analysis ---")

# --- Analyze for Opportunity ---
if opportunity_text_input.strip():
    print("\nFound text for Opportunity. Analyzing...")
    swot_paragraph = generate_swot_point(opportunity_text_input, company_name, "Opportunity")
    if swot_paragraph:
        print("\n--- Generated Opportunity Paragraph ---")
        display(Markdown(swot_paragraph))
else:
    print("\nNo input provided for Opportunity. Skipping.")

# --- Analyze for Weakness ---
if weakness_text_input.strip():
    print("\nFound text for Weakness. Analyzing...")
    swot_paragraph = generate_swot_point(weakness_text_input, company_name, "Weakness")
    if swot_paragraph:
        print("\n--- Generated Weakness Paragraph ---")
        display(Markdown(swot_paragraph))
else:
    print("\nNo input provided for Weakness. Skipping.")

print("\n\n--- Analysis Complete ---")

--- Starting SWOT Analysis ---

Found text for Opportunity. Analyzing...
Analyzing text for Opportunity regarding BASF...
Analysis complete.

--- Generated Opportunity Paragraph ---


On March 28, 2025, China Construction Bank (CCB) launched an internal financial model based on DeepSeek R1 for enhanced operational efficiency.  This model, deployed privately for internal use only, leverages the advanced capabilities of DeepSeek R1 technology. Through this internal deployment, CCB would significantly improve the accuracy and speed of its financial modeling processes. Further, this would provide CCB with a competitive advantage in the market by streamlining operations and potentially reducing costs. This initiative aligns with CCB's ongoing commitment to technological innovation and its broader goal of optimizing its financial services.


Found text for Weakness. Analyzing...
Analyzing text for Weakness regarding BASF...
Analysis complete.

--- Generated Weakness Paragraph ---


On May 22, 2025, BASF Corporation agreed to pay a $700,000 penalty to the U.S. Environmental Protection Agency (EPA) for violating the Toxic Substances Control Act (TSCA).  The EPA found that BASF failed to submit required data on hundreds of chemical substances within the mandated timeframe during the 2020 reporting period, hindering the agency’s ability to assess potential risks to public health and the environment. This settlement highlights the importance of timely and accurate chemical reporting and underscores the EPA's commitment to enforcing environmental regulations, potentially impacting the chemical industry's compliance practices.



--- Analysis Complete ---
